In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

init_notebook_mode(connected=True)
%matplotlib inline

In [ ]:
dataset = 'PU'
test_ratio = 0.2
windowSize = 5
!wget http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
!wget http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat
!wget http://www.ehu.eus/ccwintco/uploads/a/a3/Salinas_corrected.mat
!wget http://www.ehu.eus/ccwintco/uploads/f/fa/Salinas_gt.mat
!wget http://www.ehu.eus/ccwintco/uploads/e/ee/PaviaU.mat
!wget http://www.ehu.eus/ccwintco/uploads/5/50/PaviaU_gt.mat


In [ ]:
dataset='PU'

In [ ]:
def loadData(name):
    if name == 'IP':
        data = sio.loadmat('./Indian_pines_corrected.mat')['indian_pines_corrected']
        labels = sio.loadmat('./Indian_pines_gt.mat')['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat('./Salinas_corrected.mat')['salinas_corrected']
        labels = sio.loadmat('./Salinas_gt.mat')['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat('./PaviaU.mat')['paviaU']
        labels = sio.loadmat('./PaviaU_gt.mat')['paviaU_gt']
    
    return data, labels

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
X, y = loadData(dataset)

X.shape, y.shape

In [ ]:
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error,silhouette_score

In [ ]:
import pylab as plt
import numpy as np
import seaborn as sns; sns.set()

import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
K = 30 if dataset == 'IP' else 15
X,pca = applyPCA(X,numComponents=K)

X.shape

In [ ]:
X, y = createImageCubes(X, y, windowSize=5)

X.shape, y.shape

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape
print(ytest)

In [ ]:
import random
def create_pairs(images , digit_indices):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(9)]) - 1
    print(n+1)
    for d in range (9):
        for i in range(n):
            z1 , z2 = digit_indices[d][i] , digit_indices[d][i+1]
            pairs += [[images[z1] , images[z2]]]
            rand = random.randrange(1,9)
            dn = (d + rand) % 9
            z1 , z2 = digit_indices[d][i] , digit_indices[dn][i+1]
            pairs += [[images[z1] , images[z2]]]
            labels += [1,0]
    return np.array(pairs) , np.array(labels)

def create_pair_on_set(images , label):
    digit_indices = [np.where(label == i)[0] for i in range(9)]
    #print(digit_indices)
    pairs , y = create_pairs(images , digit_indices)
    y = y.astype('float32')
    return pairs , y

In [ ]:
train_pairs , train_labels = create_pair_on_set(Xtrain,ytrain)
test_pairs  , test_labels  = create_pair_on_set(Xtest,ytest)


In [ ]:
print(train_pairs.shape)

In [ ]:
from tensorflow.keras import backend as k

def euclidean_distance(vects):
    x , y = vects
    sum_square = k.sum(k.square(x - y) , axis=1 , keepdims=True)
    return k.sqrt(k.maximum(sum_square , k.epsilon()))

def euclidean_dist_output_shape(shapes):
    shape1 , shape2 = shapes
    return (shape1[0] , 1)


In [ ]:
from keras.models import Sequential


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [ ]:
def initialize_base_network():
    input = Input(shape=(5,5,15), name="base_input")
    x = Conv2D(32,3,activation='relu')(input)
    x = Conv2D(64,3,activation='relu')(x)
    x = Flatten(name="flatten_input")(x)
    x = Dense(256, activation='relu', name="first_base_dense")(x)
    x = Dropout(0.1, name="first_dropout")(x)
    x = Dense(128, activation='relu', name="second_base_dense")(x)
    x = Dropout(0.1, name="second_dropout")(x)
    x = Dense(64, activation='relu', name="third_base_dense")(x)

    return Model(inputs=input, outputs=x)




In [ ]:


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda,Conv2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import random

In [ ]:
base_network = initialize_base_network()
plot_model(base_network, show_shapes=True, show_layer_names=True, to_file='base-model.png')

In [ ]:
input_left = Input(shape=(5,5,15) , name="left_input")
vector_output1 = base_network(input_left)

input_right = Input(shape=(5,5,15) , name="right_input")
vector_output2 = base_network(input_right)

output = Lambda(euclidean_distance , name = "output_layer" , output_shape = euclidean_dist_output_shape)([vector_output1 , vector_output2])
model = Model(inputs = [input_left , input_right] , outputs = output)
plot_model(model , to_file="multi_input_main_model.png",show_shapes=True , show_layer_names=True)

In [ ]:
model.summary()

In [ ]:
def constructive_loss_with_margin(margin):
    def constructive_loss(y_true , y_pred):
        square_pred = k.square(y_pred)
        margin_square = k.square(k.maximum(margin - y_pred , 0))
        return k.mean(y_true * square_pred + (1 - y_true) * margin_square)
    return constructive_loss

In [ ]:
model.compile(optimizer="rmsprop" , loss=constructive_loss_with_margin(1))


In [ ]:

history = model.fit([train_pairs[: , 0] , train_pairs[: , 1]] , train_labels , 
    epochs=15,
    validation_data=([test_pairs[:,0] , test_pairs[:,1]] , test_labels)
)

In [ ]:
y_pred = model.predict([train_pairs[:,0] , train_pairs[:,1]])


In [ ]:
def accurecy(y_true , y_pred):
    p = y_pred.ravel() < 0.5
    return np.mean(p == y_true)

In [ ]:
print("Train Accurecy is : ",accurecy(train_labels , y_pred))


In [ ]:
y_pred = model.predict([test_pairs[:,0] , test_pairs[:,1]])


In [ ]:
print("Test Accurecy is : ",accurecy(test_labels , y_pred))
